# 리뷰 문장 단위로 split
- 참고: https://docs.likejazz.com/kss/#python

## 구글드라이브 마운트, 라이브러리 호출

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kss-3.7.0.tar.gz (42.4 MB)
  Using cached kss-3.6.4.tar.gz (42.4 MB)
  Using cached emoji-1.2.0-py3-none-any.whl (131 kB)
  Created wheel for kss: filename=kss-3.6.4-py3-none-any.whl size=42448614 sha256=d8bffa0ba36edd9e5bc33945e9086e8335e4d73264f102a58a8b596baed12286
  Stored in directory: /root/.cache/pip/wheels/32/53/7c/76bdf098c2a6c5cd4c4c29648da30ad4793d604314b2aeb26f
Successfully built kss


In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import kss            # 한국어 문장 분리
from tqdm import tqdm # 반복문 시간 예측

## 데이터셋 불러오기

In [ ]:
# 데이터셋 불러오기
train_rev= pd.read_table('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/rev_data.csv', sep=",", index_col=0)    
train_shop = pd.read_table('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/shopping_rev.csv',  sep=",", index_col=0)
predict = pd.read_table('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_crawling/total_all_review.csv',  sep=",", index_col=0)

In [ ]:
# train_rev index가 중복 부여 되어있는 것 같아서(마지막 index가 8013인데 행 개수가 8931임) 새로 index 부여
train_rev.reset_index(drop=False, inplace=True)

In [ ]:
# 불러온 데이터셋 확인
train_rev

,index,review,score,binary,trinary,review_split
0,0,친절하시고 깔끔하고 좋았습니다,5,1,2,0
1,1,조용하고 고기도 굿,5,1,2,0
2,2,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",4,1,2,0
3,3,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,4,1,2,0
4,4,고기도 맛있고 서비스는 더 최고입니다~,5,1,2,0
...,...,...,...,...,...,...
8926,8009,가짜원조... 줄서는사람 뭐지???,1,0,0,0
8927,8010,짝퉁 여긴 원조랑 아무 관계 직원 연관되게 없는 그냥 이름만 .. 가짜입니다 속지 마세요,1,0,0,0
8928,8011,가성비 별로 비싸기만함,1,0,0,0
8929,8012,별 한개주기도 아까움. 별을 줘야 등록돼서 평가함! 나 같은 사람 또 안생겼으면 좋...,1,0,0,0


In [ ]:
train_shop

,score,review,binary,trinary,review_split
0,5,배공빠르고 굿,1,2,0
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,0,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1,2,0
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,0,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,2,0
...,...,...,...,...,...
199995,2,장마라그런가!!! 달지않아요,0,0,0
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...,1,2,0
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요,1,2,0
199998,5,넘이쁘고 쎄련되보이네요~,1,2,0


In [ ]:
predict

,식당 인덱스,리뷰,review_split
0,0,저렴한 전집이지만 바로 부쳐주시니 맛은 보장됩니다. 조금씩 시켜먹기 좋게 메뉴판 구...,0
1,0,굿ㅇ,0
2,0,친절해요,0
3,0,친절해요,0
4,0,굿,0
...,...,...,...
112548,1047,생활의 달인 방송후 다음날 갔는데.. 겨우 구입했어요!!\n진짜 쫀득쫀득하니 맛있네요^^,0
112549,1047,떡이 맛있어요~,0
112550,1047,사장님도 너무친절하시고 떡도 너무 맛있어요 ㅎㅎ 아침용으로 하나씩 챙겨먹을때도있고 ...,0
112551,1047,떡이 종류가 별루없네요.직접만드는거같아요\n가래떡샀는데 따뜻하네요~,0


## 데이터셋 전처리
- kss가 '\n' 뒤 문장을 다 지워버리는 문제가 있어 기존 review에서 \n 모두 삭제


In [ ]:
# 예시
s = "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 \n 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다\n강남역 맛집 토끼정의 외부 모습."
new_s = s.replace('\n', '')
print("기존: " + s)
print("수정 후: " + new_s)

기존: 회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 
 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다
강남역 맛집 토끼정의 외부 모습.
수정 후: 회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요  다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다강남역 맛집 토끼정의 외부 모습.


In [ ]:
# train_rev
for i in tqdm(range(train_rev.shape[0])):
    train_rev.loc[i, "review"] = train_rev.loc[i, "review"].replace('\n', ' ')

100%|██████████| 8931/8931 [00:05<00:00, 1661.19it/s]


In [ ]:
# train_shop
for i in tqdm(range(train_shop.shape[0])):
    train_shop.loc[i, "review"] = train_shop.loc[i, "review"].replace('\n', ' ')

100%|██████████| 200000/200000 [52:47<00:00, 63.14it/s]


In [ ]:
# predict
for i in tqdm(range(predict.shape[0])):
    try:
        predict.loc[i, "리뷰"] = predict.loc[i, "리뷰"].replace('\n', ' ')
    except:
        pass

100%|██████████| 112553/112553 [08:47<00:00, 213.23it/s]


In [ ]:
# 일단 저장하겠습니다
train_rev.to_csv('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/train_rev.csv')
train_shop.to_csv('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/train_shop.csv')
predict.to_csv('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_crawling/predict.csv')

## 문장 단위로 분리
- 하나의 리뷰를 문장 단위로 쪼개서 리뷰 str을 문장 list로 만들기 -> review_split 열에 넣기


In [ ]:
# 위에서 저장한 데이터프레임 다시 불러오기
train_rev = pd.read_table('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/train_rev.csv', sep=",", index_col=0)    
train_shop = pd.read_table('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/train_shop.csv',  sep=",", index_col=0)
predict = pd.read_table('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_crawling/predict.csv',  sep=",", index_col=0)

In [ ]:
# 확인
train_shop

,score,review,binary,trinary,review_split
0,5,배공빠르고 굿,1,2,0
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,0,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1,2,0
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,0,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,2,0
...,...,...,...,...,...
199995,2,장마라그런가!!! 달지않아요,0,0,0
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...,1,2,0
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요,1,2,0
199998,5,넘이쁘고 쎄련되보이네요~,1,2,0


In [ ]:
# 예시
s = "회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요 다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다 강남역 맛집 토끼정의 외부 모습."
for sent in kss.split_sentences(s):
    print(sent)

회사 동료 분들과 다녀왔는데 분위기도 좋고 음식도 맛있었어요
다만, 강남 토끼정이 강남 쉑쉑버거 골목길로 쭉 올라가야 하는데 다들 쉑쉑버거의 유혹에 넘어갈 뻔 했답니다 강남역 맛집 토끼정의 외부 모습.


In [ ]:
# train_rev, train_shop, predict에 리뷰_split 열 추가
train_rev["review_split"] = ""
train_shop["review_split"] = ""
predict["review_split"] = ""

In [ ]:
# 확인
train_shop[30:40]

,score,review,binary,trinary,review_split
30,2,설치할때 전압이 넘 높아 퓨즈 나갔어여,0,0,
31,2,첨에받았을땐 너무튼튼하구 칸막이도있구 넘맘에들었는데요... 지금도맘에는들어요..근데...,0,0,
32,2,휴대폰 각이 잘 안나옴 노트9 사용 얇은 케이스 꺼도 잘 빠지지가 않네요 진짜 불편...,0,0,
33,5,너무 생각보다 고급스럽,1,2,
34,5,가볍고 좋아요. 옵션에 파우치도 있었으면 얼마나 좋았을까 아쉽네요. 따로 구매 했어...,1,2,
35,1,"총 4개 샀는데,사용하기 전에 부드러운 물티슈로 닦았는데,프린트가 번졌네요 교환하기...",0,0,
36,2,모기가 않들어가요ᆢ,0,0,
37,5,선물준사람도 받은사람도 모두 만족했어요^^,1,2,
38,1,예쁘고 심플해서 샀는데. 재질이 깔끄러워요. 살에 자국 다 베이고ㅠㅠ....폭망이에...,0,0,
39,4,고정이 잘되기는 하는데 많이 쓰면 떡진것 처럼 보일까봐 걱정되서 많이 못쓰겠어요,1,2,


### train_rev

In [ ]:
# train_rev
for i in tqdm(range(train_rev.shape[0])):
    review = train_rev.loc[i, "review"]
    review_list = []
    for sent in kss.split_sentences(review):
        review_list.append(sent)
    train_rev.at[i, "review_split"] = review_list

100%|██████████| 8931/8931 [02:48<00:00, 52.88it/s]


In [ ]:
# 확인
train_rev[30:40]

,index,review,score,binary,trinary,review_split
30,30,뼈없는 국물 닭발과 치즈계란말이. 주먹밥의 조화~ 맛집이라 그런지 여성 그룹들이 많...,4,1,2,[뼈없는 국물 닭발과 치즈계란말이. 주먹밥의 조화~ 맛집이라 그런지 여성 그룹들이 ...
31,31,친구소개로 갔는데..깜놀이네요 이런 육즙이 터지는 고기맛은 처음이네요. 인생삼겹인정...,5,1,2,"[친구소개로 갔는데.., 깜놀이네요, 이런 육즙이 터지는 고기맛은 처음이네요., 인..."
32,32,분위기도좋고 고기도맛있어요. 직원들도 친절합니다.,5,1,2,"[분위기도좋고 고기도맛있어요., 직원들도 친절합니다.]"
33,33,블로그보고 예약해서 갔어요.. 대부분 블로그는 광고가 많은데 여기 목살은 블로그에 ...,4,1,2,"[블로그보고 예약해서 갔어요.., 대부분 블로그는 광고가 많은데 여기 목살은 블로그..."
34,34,목살이 부드러울수있다니 ㅠㅠ 퍽퍽한거 일도 없이 육즙가득 JMT 바람불어도 냉면을 ...,5,1,2,[목살이 부드러울수있다니 ㅠㅠ 퍽퍽한거 일도 없이 육즙가득 JMT 바람불어도 냉면을...
35,35,정육왕 블로그 보고 왔는데 목살이 그렇게 맛있다고해서 먹었는데 먹어본 목살중에 제일...,5,1,2,[정육왕 블로그 보고 왔는데 목살이 그렇게 맛있다고해서 먹었는데 먹어본 목살중에 제...
36,36,고기는 맛있습니다. 순환율이 굉장히 좋은 곳이라 예약하기엔 많이 부담스럽네요. 주문...,2,0,0,"[고기는 맛있습니다., 순환율이 굉장히 좋은 곳이라 예약하기엔 많이 부담스럽네요.,..."
37,37,불판이 작아서 6명 테이블에 6명이 고기먹기엔 힘들었어요 ㅠㅠ,3,0,1,[불판이 작아서 6명 테이블에 6명이 고기먹기엔 힘들었어요 ㅠㅠ]
38,38,좋았습니다,5,1,2,[좋았습니다]
39,39,분위기도 좋고 했지만 고기가 얇아 아쉬웠습니다.,4,1,2,[분위기도 좋고 했지만 고기가 얇아 아쉬웠습니다.]


In [ ]:
train_rev.loc[33, "review_split"]

['블로그보고 예약해서 갔어요..',
 '대부분 블로그는 광고가 많은데 여기 목살은 블로그에 나온것처럼 진짜 맛이있네요...',
 '감칠맛이랄까...ㅋㅋ',
 '첨으로 먹어본 목살맛이었어요.,. 후회없는 선택이었습니다^^',
 '강추!!!!']

In [ ]:
# csv 파일로 저장
train_rev.to_csv('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/train_rev_split.csv')

### train_shop

In [ ]:
# train_shop
for i in tqdm(range(train_shop.shape[0])):
    review = train_shop.loc[i, "review"]
    review_list = []
    for sent in kss.split_sentences(review):
        review_list.append(sent)
    train_shop.at[i, "review_split"] = review_list

100%|██████████| 200000/200000 [53:03<00:00, 62.82it/s]


In [ ]:
# 확인
train_shop.loc[31, "review_split"]

['첨에받았을땐 너무튼튼하구 칸막이도있구 넘맘에들었는데요...',
 '지금도맘에는들어요..',
 '근데....먼지가 먼지가 세상먼지다묻어요ㅠㅠ',
 '흑흑ㅠ 집에가만히 뒀는데도 먼지가다묻어있어요ㅠ',
 '한번들고 그냥 모셔두었어요ㅜㅜ',
 '까만색은 넘나비추입니다ㅠ',
 '아이보리로살껄...ㅠ']

In [ ]:
# csv 파일로 저장
train_rev.to_csv('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_external/train_shop_split.csv')

### predict
- predict의 \\n은 왜 제거가 안됐는지 모르겠으나.. 제거 없이도 잘 되는 것 같아서 일단 그대로 가는걸로

In [ ]:
# predict
for i in tqdm(range(predict.shape[0])):
    review = predict.loc[i, "리뷰"]
    review_list = []
    try:
        for sent in kss.split_sentences(review):
            review_list.append(sent)
        predict.at[i, "review_split"] = review_list
    except:
        predict.at[i, "review_split"] = predict.at[i, "리뷰"]

100%|██████████| 112553/112553 [20:15<00:00, 92.62it/s]


In [ ]:
# 확인
predict.loc[52, "review_split"]

['밥 왜이렇게 늦게먹냐고 이럴줄 알았으면 밥 안 줬다고 주인이 역정내는집. 7시에 주문- 7시 20분에 밥나옴 - 식사 20분째부터 사장이 그릇치우기 시작\\n하더니 밥 왜이리 늦게 먹냐',
 '나 퇴근해야된다.',
 '이럴줄알았음',
 '밥 안줬다(진짜 이렇게말함)정색하고 꾸짖음ㅋㅋ 기분 잡쳐서 걍 그자리에서 계산하니까 \\n왜 남기냐고 다 먹고가라고함. ㅋㅌㅋㅋㅋ',
 '다시는 안감 진짜 장사 그렇게하지마세요',
 '퉤']

In [ ]:
predict.loc[63, "review_split"]

['곤드레밥 먹고싶을때 종종 찾습니다.',
 '반찬종류도 많고 양도 넉넉히 나오네요~\\n개인적으로 곤드레가 좀더 많이 들어가면 좋겠고 반찬이 덜 짜면 좋겠어요~',
 '\\n그래도 맛나고 배부르게 잘 먹었어요^^']

In [ ]:
# csv 파일로 저장
train_rev.to_csv('/content/drive/MyDrive/[22-2] 딥러닝개론/project/data_crawling/predict_split.csv')